# AtomsCalculators integration

[AtomsCalculators.jl](https://github.com/JuliaMolSim/AtomsCalculators.jl) is an interface
for doing standard computations (energies, forces, stresses, hessians) on atomistic
structures. It is very much inspired by the calculator objects in the
[atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html).

DFTK by default ships a datastructure called a `DFTKCalculator`,
which implements the AtomsCalculators interface. A `DFTKCalculator`
can be constructed by passing three different named tuples,
the `model_kwargs`, the `basis_kwargs` and the `scf_kwargs`.
The first two named tuples are passed as keyword arguments when constructing
the DFT model using `model_DFT` and its discretization using
the `PlaneWaveBasis`. The last one is used as keyword arguments
when running the
`self_consistent_field` function on the resulting basis to solve the problem
numerically. Thus when using the `DFTKCalculator` the user is expected to
pass these objects exactly the keyword argument one would pass when constructing
a `model` and `basis` and when calling `self_consistent_field`.

For example, to perform the calculation of the Tutorial using
the AtomsCalculators interface we define the calculator as such:

In [1]:
using DFTK
using PseudoPotentialData

pd_lda_family = PseudoFamily("dojo.nc.sr.lda.v0_4_1.oncvpsp3.standard.upf")
model_kwargs  = (; functionals=LDA(), pseudopotentials=pd_lda_family)
basis_kwargs  = (; kgrid=[4, 4, 4], Ecut=7)
scf_kwargs    = (; tol=1e-5)
calc = DFTKCalculator(; model_kwargs, basis_kwargs, scf_kwargs)

DFTKCalculator(functionals=Xc(lda_x, lda_c_pw), pseudopotentials=PseudoFamily("dojo.nc.sr.lda.v0_4_1.oncvpsp3.standard.upf"), Ecut=7, kgrid=[4, 4, 4])

Note, that the `scf_kwargs` is optional and can be missing
(then the defaults of `self_consistent_field` are used).

> **Kpoints from kpoint density**
>
> Note that DFTK's `kgrid_from_maximal_spacing` function can also be used with
> `AbstractSystem` objects to determine an appropriate `kgrid` paramter for the `basis_kwargs`.
> E.g. `kgrid_from_maximal_spacing(system, 0.25u"1/Å")` gives a k-point spacing of
> `0.25` per Angström for the passed system.

Based on this `calc` object we can perform a DFT calculation on bulk silicon
according to the
[`AtomsCalculators` interface](https://juliamolsim.github.io/AtomsCalculators.jl/stable/interface/),
e.g.

In [2]:
using AtomsBuilder
using AtomsCalculators
AC = AtomsCalculators

# Bulk silicon system of the Tutorial
silicon = bulk(:Si)
AC.potential_energy(silicon, calc)  # Compute total energy

-8.508511644208447 Eₕ

or we can compute the energy and forces:

In [3]:
results = AC.calculate((AC.Energy(), AC.Forces()), silicon, calc)
results.energy

-8.508511644210849 Eₕ

In [4]:
results.forces

2-element Vector{StaticArraysCore.SVector{3, Unitful.Quantity{Float64, 𝐋 𝐌 𝐓^-2, Unitful.FreeUnits{(a₀^-1, Eₕ), 𝐋 𝐌 𝐓^-2, nothing}}}}:
 [1.654588077858738e-14 Eₕ a₀^-1, 1.613951162940974e-14 Eₕ a₀^-1, 1.690720539396413e-14 Eₕ a₀^-1]
 [-1.6577222975825785e-14 Eₕ a₀^-1, -1.605796463714983e-14 Eₕ a₀^-1, -1.6773770231521402e-14 Eₕ a₀^-1]

Note that the `results` object returned by the call to `AtomsCalculators.calculate`
also contains a `state`, which is a DFTK `scfres`. This can be used to speed up
subsequent computations:

In [5]:
# This is basically for free, since already computed:
results2 = @time AC.calculate((AC.Energy(), AC.Forces()), silicon, calc, nothing, results.state);

  0.353615 seconds (831.52 k allocations: 87.517 MiB, 10.39% gc time, 9.65% compilation time)


For an example using the `DFTKCalculator`, see Geometry optimization.